image analysis running on the server "lev", sharing the basic same data.  presently, the IA port is not exposed, but in interest of time, i am making a "pass-through" access to the IA api via a WB api call.  (this is ultimately the model we want to use in production, so that wildbook can handle security and other issues, but as you will see, i have kind of done a very generic hack version of the same concept.)   this consists of a small wb api "wrapper" call around the call you actually wish to be making of the IA data.  perhaps examples would demonstrate best:

IA call:  /api/annot/image/contributor/tag/json/?annot_uuid_list=[{"__UUID__":"8b595dc0-9c5a-4caf-9703-9f8ff017e824"}]
becomes: http://lev.cs.rpi.edu:8080/ggr/ia?passthru=/api/annot/image/contributor/tag/json/&arg=annot_uuid_list%3D[{%22__UUID__%22:%228b595dc0-9c5a-4caf-9703-9f8ff017e824%22}]

IA call:  /api/annot/age/months/json/?annot_uuid_list=[{"__UUID__":"8b595dc0-9c5a-4caf-9703-9f8ff017e824"}]
becomes:  http://lev.cs.rpi.edu:8080/ggr/ia?passthru=/api/annot/age/months/json/&arg=annot_uuid_list%3D[{%22__UUID__%22:%228b595dc0-9c5a-4caf-9703-9f8ff017e824%22}]

in other words, you pass two parameters, passthru and arg, which are just uri-encoded strings that represent the two sides of the "?" in the original call.  (note that arg is optional.)  the two examples above are "live", in the sense that you can click them and should get the json results as expected.

this should, technically, get you to any existing IA api call (provided i had jason p turn on all the right ones)... at least the ones that use GET.  if you need any POST queries, let me know and i can pass those through as well.

In [79]:
import requests
import urllib
from collections import OrderedDict

In [ ]:
## /api/annot/image/contributor/tag/json/?annot_uuid_list=[{"__UUID__":"8b595dc0-9c5a-4caf-9703-9f8ff017e824"}]

DOMAIN = 'http://lev.cs.rpi.edu:8080/ggr//ia'

param = urllib.parse.quote('annot_uuid_list=') + '[{' + urllib.parse.quote('"__UUID__"') + ':' +urllib.parse.quote('"8b595dc0-9c5a-4caf-9703-9f8ff017e824"') + '}]'
# param = urllib.parse.urlencode(param) 

In [262]:
payload = OrderedDict([('passthru', u'/apaaaai///annot/age/months/json/'), 
                ('asdasarg', param)]) 
response = requests.get(baseurl , 
                        params = payload 
                        )

In [258]:
response.request.path_url

'/ggr/ia?passthru=%2Fapaaaai%2F%2F%2Fannot%2Fage%2Fmonths%2Fjson%2F&asdasarg=annot_uuid_list%253D%5B%7B%2522__UUID__%2522%3A%25228b595dc0-9c5a-4caf-9703-9f8ff017e824%2522%7D%5D'

In [277]:
url = DOMAIN + '?passthru=' + '/api/annot/age/months/json/' + '&arg=' + ggr_form_arg("8b595dc0-9c5a-4caf-9703-9f8ff017e824")

In [279]:
res = requests.get(url)


In [280]:
res.json()

{'results': [[6, 11]], 'success': True}

In [251]:
assert response.ok

In [267]:
s = "abcd"

'\"%s\"' %s

'"abcd"'

In [275]:
ggr_form_arg = lambda x : urllib.parse.quote('annot_uuid_list=') + '[{' + urllib.parse.quote('"__UUID__"') + ':' + urllib.parse.quote('\"%s\"' %x) + '}]'

In [1]:
import UploadAndDetectIBEIS as UD, uuid
import requests, json, datetime, urllib, importlib
importlib.reload(UD)

import GetPropertiesAPI as GP
importlib.reload(GP)

<module 'GetPropertiesAPI' from '/Users/sreejithmenon/Google Drive/CodeBase/AWESOME/script/GetPropertiesAPI.py'>

In [ ]:
data_dict = {
        'gid_list': [1],
    }
image_uuid_list = UD.get('api/image/uuid', data_dict)
image_uuid_dict = image_uuid_list[0]
image_uuid = uuid.UUID(image_uuid_dict['__UUID__'])

data_dict = {
    'query_annot_uuid_list' : [1],
    'query_annot_name_list' : [image_uuid]
    
}
res = requests.get('http://pachy.cs.uic.edu:5000/api/engine/query/graph/', data = data_dict)
res.json()

In [190]:
data_dict = {
        'aid_list': [1],
    }
image_uuid_list = UD.get('api/annot/uuid', data_dict)
image_uuid_list

http://pachy.cs.uic.edu:5001/api/annot/uuid/
{'response': [{'__UUID__': '2637b5e7-2a41-4aea-9dbd-dfe186cc8c8c'}], 'status': {'cache': -1, 'code': 200, 'message': '', 'success': True}}


[{'__UUID__': '2637b5e7-2a41-4aea-9dbd-dfe186cc8c8c'}]

In [93]:
res = requests.get('http://pachy.cs.uic.edu:5001/api/annot/uuid/', 
                   data = dict(aid_list=['1']))


print(res.json())

res = requests.get('http://pachy.cs.uic.edu:5000/api/annot/uuid/', 
                   data = dict(aid_list=['15441']))


print(res.json())

{'response': [{'__UUID__': '2637b5e7-2a41-4aea-9dbd-dfe186cc8c8c'}], 'status': {'cache': -1, 'code': 200, 'message': '', 'success': True}}
{'response': [{'__UUID__': '4abaab8a-8ff3-4142-acbb-226fcac615f1'}], 'status': {'cache': -1, 'code': 200, 'message': '', 'success': True}}


In [149]:
url = "http://pachy.cs.uic.edu:5001/api/engine/query/graph/"
data_dict = {
    'query_annot_uuid_list' : json.dumps([{'__UUID__': '2637b5e7-2a41-4aea-9dbd-dfe186cc8c8c'}]),
}
response = requests.request('POST', url, data=data_dict)

In [152]:
response.json()

{'response': 'jobid-1713',
 'status': {'cache': -1, 'code': 200, 'message': '', 'success': True}}

In [161]:
d['json_result']['inference_dict']['cluster_dict']

{'annot_uuid_list': [{'__UUID__': '50911e26-59ec-477e-adc7-20eac7208cb3'},
  {'__UUID__': '2637b5e7-2a41-4aea-9dbd-dfe186cc8c8c'},
  {'__UUID__': '814abf6d-9237-45b5-a389-2b8031eb7572'}],
 'error_flag_list': [['merge'], ['merge'], ['merge']],
 'exemplar_flag_list': [False, True, False],
 'new_name_list': ['NEWNAME_1', 'NEWNAME_1', 'NEWNAME_1'],
 'orig_name_list': ['NEWNAME_-1796', 'NEWNAME_-1', 'NEWNAME_-1353']}

In [163]:
d['json_result']['inference_dict']['annot_pair_dict']

{'confidence_list': [0.9374784921383149,
  0.9999999942117608,
  0.9999999999999716,
  1.0],
 'review_pair_list': [{'annot_uuid_1': {'__UUID__': '2637b5e7-2a41-4aea-9dbd-dfe186cc8c8c'},
   'annot_uuid_2': {'__UUID__': '843e805c-3df9-4b1a-b2b3-6e363aa1f92e'},
   'annot_uuid_key': {'__UUID__': '2637b5e7-2a41-4aea-9dbd-dfe186cc8c8c'},
   'prior_matching_state': {'p_match': 0.9841173649380682,
    'p_nomatch': 0.01588263506193177,
    'p_notcomp': 0.0}},
  {'annot_uuid_1': {'__UUID__': '2637b5e7-2a41-4aea-9dbd-dfe186cc8c8c'},
   'annot_uuid_2': {'__UUID__': '7b8b2967-beab-4399-b572-d70faeda7b33'},
   'annot_uuid_key': {'__UUID__': '2637b5e7-2a41-4aea-9dbd-dfe186cc8c8c'},
   'prior_matching_state': {'p_match': 0.9999999985529402,
    'p_nomatch': 1.4470598053151207e-09,
    'p_notcomp': 0.0}},
  {'annot_uuid_1': {'__UUID__': '2637b5e7-2a41-4aea-9dbd-dfe186cc8c8c'},
   'annot_uuid_2': {'__UUID__': '50911e26-59ec-477e-adc7-20eac7208cb3'},
   'annot_uuid_key': {'__UUID__': '2637b5e7-2a41-4aea-

In [244]:
data_dict = {
    "aid_list" : ['5'],
    "name_list" : ['2']
}
d = UD.put("api/annot/name", data_dict)

In [223]:
def run_id_pipeline(aid): # ID'ing task for each annotation
    # step 1: get annot_uuid
    data_dict = {
        'aid_list': [aid],
    }
    annot_uuid_list = UD.get('api/annot/uuid', data_dict)

    # step 2: for the given annot UUID run the detection against all the available annots
    data_dict = {
        'query_annot_uuid_list' : json.dumps([annot_uuid_list[0]])
    }
    response = requests.request('POST', url, data=data_dict)

    try:
        assert response.json()['status']['success']
    except AssertionError:
        print("RUN_ID_PIPELINE failed for annotation_id %i" %aid)

    jobid_str = response.json()['response']

    while not UD.check_job_status(jobid_str):
        print("Waiting..")
        time.sleep(5)

    # step 3: Job execution must have successfully completed at this point and now we extract the needed information
    data_dict = {
        'jobid' : jobid_str
    }

    result = UD.get("api/engine/job/result", data_dict)['json_result']['inference_dict']['cluster_dict']


    return result

In [224]:
r = run_id_pipeline(1)

Waiting..


In [200]:
json.dumps([{'__UUID__': '2637b5e7-2a41-4aea-9dbd-dfe186cc8c8c'}])

'[{"__UUID__": "2637b5e7-2a41-4aea-9dbd-dfe186cc8c8c"}]'

In [227]:
r['orig_name_list']

['NEWNAME_-1796', '1', 'NEWNAME_-1353']

In [208]:
req_status = r.json()['status']['success']

In [216]:
time.sleep(5)

In [215]:
import time

In [43]:
data_dict = {
    'name_rowid_list' : [1]
}


UD.delete('api/name', data_dict)

In [40]:
def run_id_pipeline(gidRange):
    gid_aid_map = {}

    for gid in gidRange:
        aid = GP.getAnnotID(int(gid))
        gid_aid_map[gid] = [aid][0]

    aid_list = [item for sublist in gid_aid_map.values() for item in sublist if len(sublist) > 0] # flatten out the list of lists, exclude ones with no annotation

    len(aid_list)
    for aid in aid_list:
        print("Running ID detection for aid %s" %aid)

In [41]:
run_id_pipeline(range(1,1702))

Running ID detection for aid 2047


In [12]:
with open("../data/fileURLS_old.dat", "r") as old:
    old_l = old.read().split("\n")
    
with open("../data/fileURLS.dat", "r") as new:
    new_l = new.read().split("\n")

In [17]:
new_l = list(set(new_l) - set(old_l))


with open("../data/fileURLs.dat", "w") as new:
    new.write("\n".join(new_l))

In [42]:
GP.getAnnotID(int(gid))[0]

2047

### Extracting GID-AID map and then UUID : AID map 

In [19]:
allGID = list(range(1,1702))

GidAidMap = {}
for gid in allGID:
    aid = GP.getAnnotID(int(gid))
    GidAidMap[gid] = [aid][0]

In [22]:
GidAidMap = {gid : GidAidMap[gid][0] for gid in GidAidMap.keys()}

In [28]:
aid_list = [item for sublist in GidAidMap.values() for item in sublist if len(sublist) > 0]

UUID = GP.getImageFeature(aid_list, "uuid")

In [34]:
uuid_aid_map = {UUID[i]["__UUID__"] : aid_list[i] for i in range(len(UUID))}

In [31]:
len(UUID), len(aid_list)

(2047, 2047)

In [36]:
with open("Flick_UUID_AID_map.json", "w") as mapFl:
    json.dump(uuid_aid_map, mapFl, indent=4)

In [46]:
def run_id_detection(aid): # ID'ing task for each annotation
    # step 1: get annot_uuid -- this can be done offline too
    data_dict = {
        'aid_list': [aid],
    }
    annot_uuid_list = UD.get('api/annot/uuid', data_dict)

    # step 2: for the given annot UUID run the detection against all the available annots
    url = "http://pachy.cs.uic.edu:5001/api/engine/query/graph/"
    data_dict = {
        'query_annot_uuid_list' : json.dumps([annot_uuid_list[0]])
    }
    response = requests.request('POST', url, data=data_dict)

    print("Query submitted..!")

    try:
        assert response.json()['status']['success']
    except AssertionError:
        print("RUN_ID_PIPELINE failed for annotation_id %i" %aid)

    jobid_str = response.json()['response']
    print("Job ID: %s" %jobid_str)

    error_time = 300
    start = 0
    while not UD.check_job_status(jobid_str) and start < error_time:
        print("Waiting for job completion..!")
        start += 5
        time.sleep(5)

    try:
        assert check_job_status(jobid_str)
    except AssertionError:
        print("RUN_ID_PIPELINE failed for annotation_id %i" %aid)

    print("Query complete..!")

    # step 3: Job execution must have successfully completed at this point and now we extract the needed information
    data_dict = {
        'jobid' : jobid_str
    }

    result = get("api/engine/job/result", data_dict)['json_result']['inference_dict']['cluster_dict']
    return result
    with open("../data/Flick_UUID_AID_map.json", "r") as uuid_aid_map_json:
        uuid_aid_map = json.load(uuid_aid_map_json)

    # check if there is a previously assigned name, if no name is assigned just assign the new_name_"n"
    name = None
    for i in range(len(result['orig_name_list'])):
        if 'NEWNAME' not in result['orig_name_list'][i]:
            # this i has been previously assigned a name
            name = result['orig_name_list'][i]

    if name == None:
        name = re.findall(r'NEWNAME_(\d+)', result['new_name_list'][0])[0]

    # make the final assignment
    for annot_uuid_dict in result['annot_uuid_list']:
        aid = uuid_aid_map[annot_uuid_dict["__UUID__"]]

        data_dict = {
            "aid_list" : [aid],
            "name_list" : [name]
        }

        r = put("api/annot/name", data_dict)
    
    print("IDing complete for AID %s \n" %aid)
    return 0

In [47]:
run_id_detection(6)

Query submitted..!
Job ID: jobid-1735


NameError: name 'check_job_status' is not defined

In [49]:
data_dict = {
        'jobid' : 'jobid-1735'
}

result = UD.get("api/engine/job/result", data_dict)['json_result']['inference_dict']['cluster_dict']


In [50]:
result

{'annot_uuid_list': [{'__UUID__': '0fcfd22d-4e1d-437e-8629-366ec6ce1342'}],
 'error_flag_list': [[]],
 'exemplar_flag_list': [True],
 'new_name_list': ['NEWNAME_-6'],
 'orig_name_list': ['NEWNAME_-6']}

In [55]:
for i in range(len(result['orig_name_list'])):
    if 'NEWNAME' not in result['orig_name_list'][i]:
     # this i has been previously assigned a name
        name = result['orig_name_list'][i]
        
import re

In [57]:
name = re.findall(r'NEWNAME_(\d+)', result['new_name_list'])[0]

TypeError: expected string or bytes-like object

In [59]:
result['new_name_list'][0]

'NEWNAME_-6'

In [60]:
re.findall(r'NEWNAME_(\d+)', 'NEWNAME_-6')

[]

In [62]:
import os

In [63]:
os.path.basename('filname.txt')

'filname.txt'